In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

plt.rcParams.update({'font.size': 18})

In [ ]:
# Setup simulation parameters and define the input voltage for different 
# conditions (step and sinusoidal)
t_step = 0.000001
sim_time = 0.01   # We will be simulating a waveform from t=
n_single_sided_steps = int(sim_time/t_step)
n_steps = 2*n_single_sided_steps + 1
half_pulse_width = int(0.5 * n_single_sided_steps)
tpts = np.linspace(-sim_time, sim_time, n_steps)
omega = 1000
vs_val = 12

vt_step = np.zeros(n_steps, dtype='float')
vt_step[n_single_sided_steps+1:] = vs_val

vt_pulse = np.zeros(n_steps, dtype='float')
vt_pulse[n_single_sided_steps+1-half_pulse_width:n_single_sided_steps+half_pulse_width] = vs_val

vt_sine_wvf = vs_val * np.cos(omega * tpts)

# Set up the circuit parameters
R1 = 1e1
R2 = 1e1
C = 24e-6
L = 5e-3

plt.plot(1000 * tpts, vt_step, lw=2.0)
plt.xlabel('Time (ms)')
plt.ylabel('$v_{s}(t)$ (Volts)')
plt.grid(True)
# plt.title('Step input')
plt.show()

plt.plot(1000 * tpts, vt_pulse,  lw=2.0)
plt.xlabel('Time (ms)')
plt.ylabel('$v_{s}(t)$ (Volts)')
plt.grid(True)
# plt.title('Pulse input')
plt.show()

In [ ]:
# Here we will construct our system of equations for the circuit
A = np.zeros((2,2), dtype='float')
B = np.zeros((2,1), dtype='float')

# Fill out the different entries of matrices A and B from the problem.
# =================== Your code goes here ===================== #
A[0,0] = 0
A[0,1] = 0
A[1,0] = 0
A[1,1] = 0

B[0,0] = 0
B[1,0] = 0
# ===================  End of code block  ===================== #

xt_init = np.zeros(2, dtype='float')

# Next, we need to specify the intial conditions for state variables
# =================== Your code goes here ===================== #
xt_init[0] = 0
xt_init[1] = 0
# ===================  End of code block  ===================== #

# First, as a sanity check, let's calculate the eigenvalues of A
[eig_values, eig_vectors] = np.linalg.eig(A)
print("Eigenvalues for the matrix A are")
print(eig_values[0])
print(eig_values[1])

In [12]:
# Code for simulating the step input
# You do not need to make any changes to this code. It performs a numerical 
# integration of the differential equation.
def matrix_diffeq_dx_dt_v_step(t,xt):
  if t > 0:
    v_step = 12
  else:
    v_step = 0
  dx_dt = np.matmul(A, xt.reshape((-1,1))) + B * v_step
  return dx_dt.reshape((1,-1))

def matrix_diffeq_dx_dt_v_pulse(t,xt):
  if t < -0.005:
    v_pulse = 0
  elif t < 0.005:
    v_pulse = 12
  else:
    v_pulse  = 0
  dx_dt = np.matmul(A, xt.reshape((-1,1))) + B * v_pulse
  return dx_dt.reshape((1,-1))

def matrix_diffeq_dx_dt_v_sine(t,xt):
  dx_dt = np.matmul(A, xt.reshape((-1,1))) + B * vs_val * np.cos(omega * t)
  return dx_dt.reshape((1,-1))

In [ ]:
# Having defined the derivative functions, we can use a numerical tool to solve
# the differential equation with the initial condition

pulse_input_sol = solve_ivp(matrix_diffeq_dx_dt_v_pulse, \
                           [tpts[0], tpts[-1]], \
                           xt_init, t_eval=tpts)
plt.plot(1000 * pulse_input_sol.t, pulse_input_sol.y[1,:], lw=2.0)
plt.xlabel('Time (ms)')
plt.ylabel('$v_{out}(t)$ (Volts)')
plt.grid(True)

In [ ]:
step_input_sol = solve_ivp(matrix_diffeq_dx_dt_v_step, \
                           [tpts[0], tpts[-1]], \
                           xt_init, t_eval=tpts)
plt.plot(1000 * step_input_sol.t, step_input_sol.y[1,:])
plt.xlabel('Time (ms)')
plt.ylabel('$v_{out}(t)$ (Volts)')
plt.grid(True)

In [ ]:
sine_input_sol = solve_ivp(matrix_diffeq_dx_dt_v_sine, \
                           [tpts[0], tpts[-1]], \
                           xt_init, t_eval=tpts)
plt.plot(1000 * sine_input_sol.t, sine_input_sol.y[1,:], lw=2.0, label='$v_{out}$')
plt.plot(1000 * tpts, vt_sine_wvf, lw=2.0, label='$v_{s}$')
plt.xlabel('Time (ms)')
plt.ylabel('$v_{out}(t)$ (Volts)')
plt.legend()
plt.grid(True)